# This Notebook

In this notebook we will be writing a function to find the location of amino acids modified by Post-Translational Modification (PTM) in a full protein sequence given the location of the amino acid in a peptide fragement. Given the file `input.tsv` containing a list of peptide fragments, some containing PTMs, and a file `proteins.fasta` containing named full protein sequences, the function outputs a `csv` file with three columns; `ProteinName`, the protein in which the PTM is found, `FullPeptideName`, the full peptide fragment including information on the modded amino acid, and `ModifiedAminoPositions`, the zero-indexed indices of the modifed amino acids in then full protein sequence.

*Note: This project intentionally uses base python only, though importing modules like csv, re (RegEx), Pandas, and others may have been helpful.*

## Exploring Files

From manually opening the file, some don't have PTMs, so we'll keep only the lines that have 'Unimod' when we read it in.

In [1]:
peptidesPTM = []
with open('./data/input.tsv') as f:
    for line in f:
        if 'UniMod' in line:
            peptidesPTM.append(line)

In [2]:
peptidesPTM[:5]

['DN(UniMod:5)SVHWERPQKPK\n',
 'DVFFG(UniMod:6)PK\n',
 'FKDLGEEN(UniMod:5)FK\n',
 'IQHC(UniMod:4)KKLSSWVLLM(UniMod:35)K\n',
 'L(UniMod:7)AEYHAKATEHLSTLSEK\n']

Oops, need to remove new line indicator.

In [3]:
peptidesPTM = [peptide.rstrip() for peptide in peptidesPTM]

In [4]:
peptidesPTM[:5]

['DN(UniMod:5)SVHWERPQKPK',
 'DVFFG(UniMod:6)PK',
 'FKDLGEEN(UniMod:5)FK',
 'IQHC(UniMod:4)KKLSSWVLLM(UniMod:35)K',
 'L(UniMod:7)AEYHAKATEHLSTLSEK']

Looks nice, now here and from looking at the file I see we'll need to be able to handle peptide fragments with multiple modified amino acids. I'm thinking splitting the fragments at the modified acids might be a good strategy.

In [5]:
test_peptide = peptidesPTM[3]

In [6]:
test_peptide.find('HC')

2

In [7]:
search_strs = test_peptide.split('(')
for idx, search_str in enumerate(search_strs):
    if 'Uni' in search_str:
        close_loc = search_str.find(')')
        search_strs[idx] = search_str[close_loc+1:]        

In [8]:
search_strs

['IQHC', 'KKLSSWVLLM', 'K']

Let's bring in the protein sequences to see if we can work with the split fragments.

In [9]:
proteins = {}
with open('./data/proteins.fasta') as f:
    for line in f:
        if '>' in line:
            line = line.replace('>','').rstrip()
            current_protein = line
            proteins[current_protein] = ''
        else:
            proteins[current_protein] += line.rstrip()

Testing to see if the proteins match a copy paste from the fasta file.

In [10]:
proteins['O75711'] == 'MKLMVLVFTIGLTLLLGVQAMPANRLSCYRKILKDHNCHNLPEGVADLTQIDVNVQDHFWDGKGCEMICYCNFSELLCCPKDVFFGPKISFVIPCNNQ'

True

In [11]:
for protein,sequence in proteins.items():
    if 'DVFFGPK' in sequence:
        print(f'Found in {protein}!')

Found in O75711!


In [12]:
proteins['O75711'].find('DVFFG')

81

In [13]:
proteins['O75711'][85]

'G'

Ok so for the index of G we need the `find` index plus `len(fragment) - 1`. Let's try maybe splitting again?

In [14]:
this_prot_sections = proteins['O75711'].split('DVFFG')
this_prot_sections

['MKLMVLVFTIGLTLLLGVQAMPANRLSCYRKILKDHNCHNLPEGVADLTQIDVNVQDHFWDGKGCEMICYCNFSELLCCPK',
 'PKISFVIPCNNQ']

In [15]:
len(this_prot_sections[0])

81

In [16]:
this_prot_sections[0][80]

'K'

In [17]:
for section in this_prot_sections:
    print(section.find('PK')) 

79
0


In [18]:
proteins['O75711'].find('DVFFGPK')

81

Hmm.. So trying these few blocks for a few of the first five peptide fragments, I'm seeing that joining the peptide fragments without the mod info and searching for it in the protein is the most simple solution. Shorter peptide fragment sections may occur elsewhere in the protein, and not necessarily in conjuction with the rest of the full peptide. But we need to handle finding the location of multiple modified acids in the fragments, so I'm gonna explore a test solution.

In [19]:
''.join(['h','e','l'])

'hel'

We'll imagine that 'h' and 'l' are modified. I think find the last one, grab the positions, then shorten the string until we can't find the desired fragment anymore. This will make maintaining the proper index easy compared to removing the first index and shortening, or looping through characters.

In [20]:
test_str = 'aksldjhelasdasdlkjsdhelasdjkldfg'

In [21]:
test_str.rfind('hel')

20

In [22]:
test_str[:20]

'aksldjhelasdasdlkjsd'

In [23]:
locations = []
chars = ['h','e','l']
locs_in_peptide = [0,2]
while ''.join(chars) in test_str:
    start = test_str.rfind(''.join(chars))
    for loc in locs_in_peptide:
        print(start + loc)
    test_str = test_str[:start]

20
22
6
8


Okay, that seems to work. Let's put all the pieces together to prepare for our final function.

### Helper Functions

To keep the production function clean and easy to read, we'll prepare some helper functions. First we need a dictionary of the split peptide fragments with their full peptide name, since we'll need to retain that for the final output.

In [24]:
def read_peptides(peptide_tsv_filepath):
    """
    Takes in a tsv file of peptide fragments and grabs the ones that have modified amino
    acids. Returns a dictionary of the full, original peptide fragment paired with a
    list containing sections of the peptide separated upon a modified amino acid.
    """
    # Read in data from the input file and keep lines containing modded aminos,
    # designated by (UniMod:XX) where XX is a number, and clean newline
    peptidesPTM = []
    with open('./data/input.tsv') as f:
        for line in f:
            if 'UniMod' in line:
                peptidesPTM.append(line.rstrip())
    
    # To get the acid sequences without the (UniMod:XX), split on '(' then
    # remove 'UniMod:XX)' from the beginning of the following section
    peptides_dict = {}
    for peptide in peptidesPTM:
        search_strs = peptide.split('(')
        for idx, search_str in enumerate(search_strs):
            if 'Uni' in search_str:
                close_loc = search_str.find(')')
                search_strs[idx] = search_str[close_loc+1:]
        peptides_dict.update({peptide: search_strs})
    
    return peptides_dict

In [25]:
peptides_dict = read_peptides('./data/input.tsv')

In [26]:
peptides_dict

{'DN(UniMod:5)SVHWERPQKPK': ['DN', 'SVHWERPQKPK'],
 'DVFFG(UniMod:6)PK': ['DVFFG', 'PK'],
 'FKDLGEEN(UniMod:5)FK': ['FKDLGEEN', 'FK'],
 'IQHC(UniMod:4)KKLSSWVLLM(UniMod:35)K': ['IQHC', 'KKLSSWVLLM', 'K'],
 'L(UniMod:7)AEYHAKATEHLSTLSEK': ['L', 'AEYHAKATEHLSTLSEK'],
 'L(UniMod:7)DGKFSVVYAK': ['L', 'DGKFSVVYAK'],
 'L(UniMod:7)G(UniMod:6)G(UniMod:6)HLDAK': ['L', 'G', 'G', 'HLDAK'],
 'L(UniMod:7)SKELQAAQAR': ['L', 'SKELQAAQAR'],
 'N(UniMod:5)KFDPSLTQR': ['N', 'KFDPSLTQR'],
 'AL(UniMod:7)HPEEDPEGR': ['AL', 'HPEEDPEGR'],
 'ITGKPIKL(UniMod:7)TQVEHR': ['ITGKPIKL', 'TQVEHR'],
 'IYISG(UniMod:6)MAPRPSLAKK': ['IYISG', 'MAPRPSLAKK'],
 'IHWESASL(UniMod:7)L(UniMod:7)R': ['IHWESASL', 'L', 'R'],
 'ALHPEEDPEG(UniMod:6)R': ['ALHPEEDPEG', 'R'],
 'FKDL(UniMod:7)GEENFKALVL(UniMod:7)IAFAQYLQQC(UniMod:4)PFEDHVK': ['FKDL',
  'GEENFKALVL',
  'IAFAQYLQQC',
  'PFEDHVK'],
 'ILGGHL(UniMod:7)DAK': ['ILGGHL', 'DAK'],
 'AL(UniMod:7)HPEEDPEGRQGR': ['AL', 'HPEEDPEGRQGR'],
 'ITG(UniMod:6)KPIKL(UniMod:7)TQVEHR': ['ITG', '

Okay, now we'll need to do the same for proteins, a dictionary of the full name with its sequence.

In [27]:
def read_proteins(protein_fasta):
    """
    Takes in a file of protein sequences in the standard fasta format and returns
    a dictionary where the protein name is paired with the full sequence.
    """
    # Read in protein data. Lines containing '>' give a protein name, followed
    # by one or more lines with letter based amino acid sequences, so add them to
    # the sequence until the reader comes upon a new protein name.
    proteins = {}
    with open('./data/proteins.fasta') as f:
        for line in f:
            if '>' in line:
                line = line.replace('>','').rstrip()
                current_protein = line
                proteins[current_protein] = ''
            else:
                proteins[current_protein] += line.rstrip()
    return proteins

In [28]:
proteins_dict = read_proteins('./data/proteins.fasta')

In [29]:
proteins_dict

{'O75711': 'MKLMVLVFTIGLTLLLGVQAMPANRLSCYRKILKDHNCHNLPEGVADLTQIDVNVQDHFWDGKGCEMICYCNFSELLCCPKDVFFGPKISFVIPCNNQ',
 'P00738': 'MSALGAVIALLLWGQLFAVDSGNDVTDIADDGCPKPPEIAHGYVEHSVRYQCKNYYKLRTEGDGVYTLNDKKQWINKAVGDKLPECEADDGCPKPPEIAHGYVEHSVRYQCKNYYKLRTEGDGVYTLNNEKQWINKAVGDKLPECEAVCGKPKNPANPVQRILGGHLDAKGSFPWQAKMVSHHNLTTGATLINEQWLLTTAKNLFLNHSENATAKDIAPTLTLYVGKKQLVEIEKVVLHPNYSQVDIGLIKLKQKVSVNERVMPICLPSKDYAEVGRVGYVSGWGRNANFKFTDHLKYVMLPVADQDQCIRHYEGSTVPEKKTPKSPVGVQPILNEHTFCAGMSKYQEDTCYGDAGSAFAVHDLEEDTWYATGILSFDKSCAVAEYGVYVKVTSIQDWVQKTIAEN',
 'P00739': 'MSDLGAVISLLLWGRQLFALYSGNDVTDISDDRFPKPPEIANGYVEHLFRYQCKNYYRLRTEGDGVYTLNDKKQWINKAVGDKLPECEAVCGKPKNPANPVQRILGGHLDAKGSFPWQAKMVSHHNLTTGATLINEQWLLTTAKNLFLNHSENATAKDIAPTLTLYVGKKQLVEIEKVVLHPNYHQVDIGLIKLKQKVLVNERVMPICLPSKNYAEVGRVGYVSGWGQSDNFKLTDHLKYVMLPVADQYDCITHYEGSTCPKWKAPKSPVGVQPILNEHTFCVGMSKYQEDTCYGDAGSAFAVHDLEEDTWYAAGILSFDKSCAVAEYGVYVKVTSIQHWVQKTIAEN',
 'P01008': 'MYSNVIGTVTSGKRKVYLLSLLLIGFWDCVTCHGSPVDICTAKPRDIPMNPMCIYRSPEKKATEDEGSEQKIPEATNRRVWELSKANSRFA

Now, from the two dictionaries, we want to prepare a structure containing the information that will be written to our output file.

In [30]:
def get_PTM_info(peptides_dict,proteins_dict):
    """
    Takes in a peptides_dict, format {FullPeptideName: AminoSections} and
    proteins_dict, format {ProteinName: ProteinSequence} and returns a list
    [ProteinName,FullPeptideName,ModifiedAminoPositions] where the third item
    is a list containing the indices of where all the modified amino acids from
    FullPeptideName are found in ProteinName.
    """
    # For each protein and its sequence, check whether any of the peptide
    # fragments are found.
    PTM_info_list = []
    for protein,sequence in proteins_dict.items():
        for peptide,acid_secs in peptides_dict.items():
            # Create list of locations of modified acids in peptide fragment
            mod_idx = -1
            locations = []
            for s in acid_secs[:-1]:
                mod_idx += len(s)
                locations.append(mod_idx)
            # If full peptide fragment is found in the protein, using the
            # locations list, find the modified acid indices and add to info.
            # Start from end and work backwards.
            sequence_idx = []
            full_pep = ''.join(acid_secs)
            while full_pep in sequence:
                print(f'Found {peptide} in {protein}!')
                start = sequence.rfind(full_pep)
                for loc in locations:
                    sequence_idx.append(start + loc)
                sequence = sequence[:start]
            if sequence_idx != []:
                PTM_info_list.append([protein,peptide,sequence_idx])
            
    return PTM_info_list

*Note: I start mod_idx as -1, since using len(s) - 1 for fragments with multiple modded aminos would not return the proper output. This works out since even if the first amino acid has the mod, it will have len(1) resetting the starting idx to zero.*

In [31]:
info_list = get_PTM_info(peptides_dict,proteins_dict)

Found DVFFG(UniMod:6)PK in O75711!
Found L(UniMod:7)G(UniMod:6)G(UniMod:6)HLDAK in P00738!
Found L(UniMod:7)G(UniMod:6)G(UniMod:6)HLDAK in P00739!
Found FRIEDGFSL(UniMod:7)K in P01008!
Found IQHC(UniMod:4)KKLSSWVLLM(UniMod:35)K in P01009!
Found DN(UniMod:5)SVHWERPQKPK in P01023!
Found IHWESASL(UniMod:7)L(UniMod:7)R in P01024!
Found L(UniMod:7)AEYHAKATEHLSTLSEK in P02647!
Found L(UniMod:7)SKELQAAQAR in P02649!
Found L(UniMod:7)DGKFSVVYAK in P02765!
Found FKDLGEEN(UniMod:5)FK in P02768!
Found IYISG(UniMod:6)MAPRPSLAKK in P04004!
Found IIYKEN(UniMod:5)ERFQYK in P08603!
Found N(UniMod:5)KFDPSLTQR in P08697!
Found ITGKPIKL(UniMod:7)TQVEHR in P36955!
Found IHL(UniMod:7)MAGR in Q5T013!
Found AL(UniMod:7)HPEEDPEGR in Q96GW7!


Left this print out in the function though it was originally for debugging since it's kind of cool to see, but if it was processing much larger amount of information we'd probably remove it not to be printing out tons of statements.

In [32]:
info_list

[['O75711', 'DVFFG(UniMod:6)PK', [85]],
 ['P00738', 'L(UniMod:7)G(UniMod:6)G(UniMod:6)HLDAK', [162, 163, 164]],
 ['P00739', 'L(UniMod:7)G(UniMod:6)G(UniMod:6)HLDAK', [104, 105, 106]],
 ['P01008', 'FRIEDGFSL(UniMod:7)K', [362]],
 ['P01009', 'IQHC(UniMod:4)KKLSSWVLLM(UniMod:35)K', [255, 265]],
 ['P01023', 'DN(UniMod:5)SVHWERPQKPK', [1178]],
 ['P01024', 'IHWESASL(UniMod:7)L(UniMod:7)R', [1317, 1318]],
 ['P02647', 'L(UniMod:7)AEYHAKATEHLSTLSEK', [212]],
 ['P02649', 'L(UniMod:7)SKELQAAQAR', [110]],
 ['P02765', 'L(UniMod:7)DGKFSVVYAK', [120]],
 ['P02768', 'FKDLGEEN(UniMod:5)FK', [41]],
 ['P04004', 'IYISG(UniMod:6)MAPRPSLAKK', [357]],
 ['P08603', 'IIYKEN(UniMod:5)ERFQYK', [229]],
 ['P08697', 'N(UniMod:5)KFDPSLTQR', [242]],
 ['P36955', 'ITGKPIKL(UniMod:7)TQVEHR', [352]],
 ['Q5T013', 'IHL(UniMod:7)MAGR', [106]],
 ['Q96GW7', 'AL(UniMod:7)HPEEDPEGR', [879]]]

Looks like the desired output, let's test a few cases to verify the returned indices are correct.

In [33]:
proteins_dict['P00739']

'MSDLGAVISLLLWGRQLFALYSGNDVTDISDDRFPKPPEIANGYVEHLFRYQCKNYYRLRTEGDGVYTLNDKKQWINKAVGDKLPECEAVCGKPKNPANPVQRILGGHLDAKGSFPWQAKMVSHHNLTTGATLINEQWLLTTAKNLFLNHSENATAKDIAPTLTLYVGKKQLVEIEKVVLHPNYHQVDIGLIKLKQKVLVNERVMPICLPSKNYAEVGRVGYVSGWGQSDNFKLTDHLKYVMLPVADQYDCITHYEGSTCPKWKAPKSPVGVQPILNEHTFCVGMSKYQEDTCYGDAGSAFAVHDLEEDTWYAAGILSFDKSCAVAEYGVYVKVTSIQHWVQKTIAEN'

In [34]:
proteins_dict['P00739'][104]

'L'

In [35]:
proteins_dict['P00739'][105]

'G'

In [36]:
proteins_dict['P00739'][106]

'G'

In [37]:
proteins_dict['P01009'][255]

'C'

In [38]:
proteins_dict['P01009'][265]

'M'

Sweet! Now just a function to write the info to a csv file.

In [39]:
def write_to_csv(PTM_info_list,output_file_path):
    """
    Takes in a structure containing [ProteinName,FullPeptideName,ModifiedAminoPositions]
    and writes it to a csv file at the designated location, including a header.
    """
    with open(output_file_path,mode = 'w') as f:
        f.write('ProteinName,FullPeptideName,ModifiedAminoPositions\n')
        for info in PTM_info_list:
            idx_as_strs = [ str(num) for num in info[2]]
            locations = ';'.join(idx_as_strs)
            f.write(f'{info[0]},{info[1]},{locations}\n')
            
    return

In [40]:
write_to_csv(info_list,'./data/output.csv')

Output file looks good.

## A Final Function

A function that could be used in production to get PTM locations in protein sequences.

In [41]:
def find_PTMs(peptide_tsv_filepath,protein_fasta_filepath,output_filepath):
    """
    Takes in filepaths for a .tsv file containing peptide fragments and a .fasta
    file containing protein names and sequences in fasta format, as well as a
    path for the output file. The output file is a .csv file with columns
    [ProteinName,FullPeptideName,ModifiedAminoPositions] such that a row contains
    which FullPeptideName is found in ProteinName, and where the 
    ModifiedAminoPositions are found in the protein sequence of ProteinName.
    """
    peptides_dict = read_peptides(peptide_tsv_filepath)
    proteins_dict = read_proteins(protein_fasta_filepath)
    
    PTM_info_list = get_PTM_info(peptides_dict,proteins_dict)
    
    write_to_csv(PTM_info_list,output_filepath)
    
    print(f'Saved file to {output_filepath}')
    
    return            

In [42]:
find_PTMs('./data/input.tsv','./data/proteins.fasta','./data/output.csv')

Found DVFFG(UniMod:6)PK in O75711!
Found L(UniMod:7)G(UniMod:6)G(UniMod:6)HLDAK in P00738!
Found L(UniMod:7)G(UniMod:6)G(UniMod:6)HLDAK in P00739!
Found FRIEDGFSL(UniMod:7)K in P01008!
Found IQHC(UniMod:4)KKLSSWVLLM(UniMod:35)K in P01009!
Found DN(UniMod:5)SVHWERPQKPK in P01023!
Found IHWESASL(UniMod:7)L(UniMod:7)R in P01024!
Found L(UniMod:7)AEYHAKATEHLSTLSEK in P02647!
Found L(UniMod:7)SKELQAAQAR in P02649!
Found L(UniMod:7)DGKFSVVYAK in P02765!
Found FKDLGEEN(UniMod:5)FK in P02768!
Found IYISG(UniMod:6)MAPRPSLAKK in P04004!
Found IIYKEN(UniMod:5)ERFQYK in P08603!
Found N(UniMod:5)KFDPSLTQR in P08697!
Found ITGKPIKL(UniMod:7)TQVEHR in P36955!
Found IHL(UniMod:7)MAGR in Q5T013!
Found AL(UniMod:7)HPEEDPEGR in Q96GW7!
Saved file to ./data/output.csv


Sweet!